In [6]:
%%capture --no-stderr
%pip uninstall pydantic --yes
%pip install langchain_openai langchain_core langgraph langgraph_supervisor --upgrade --verbose --force-reinstall --index-url=https://pypi.org/simple/


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("AZURE_OPENAI_API_KEY")

In [13]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint="https://eastus.api.cognitive.microsoft.com/",
    azure_deployment="gpt-4o-mini",
    openai_api_version="2024-12-01-preview"
)


In [14]:
import datetime
from langgraph.prebuilt import create_react_agent

def shopopeninghours() -> str:
    "This tool provides the opening hours schedule of a shop."
    schedule = """
    Monday: 9:00 AM - 5:00 PM
    Tuesday: 9:00 AM - 5:00 PM
    Wednesday: 9:00 AM - 5:00 PM
    Thursday: 9:00 AM - 5:00 PM
    Friday: 9:00 AM - 5:00 PM
    Saturday: 10:00 AM - 4:00 PM
    Sunday: Closed
    
    May 1st is a holiday on account of Labor day.
    """
    return schedule

supervisor_agent = create_react_agent(
    tools=[shopopeninghours],
    model=llm,
    name = "supervisor_agent",
    prompt = "You are a agent that acts as a shoe store supervisor that can answer questions about the store's opening hours. "
)

def CurrentTime() -> str:
    "This tool provides the current date and time."
    return "The current date is {} and time is {}".format(datetime.date.today(), datetime.datetime.now().strftime("%H:%M:%S"))

time_teller_agent = create_react_agent(
    tools=[CurrentTime],
    model=llm,
    name = "time_teller_agent",
    prompt = "You are a agent that acts as a time teller that can answer questions about the current date and time. "
)

In [15]:
from langgraph_supervisor import create_supervisor

prompt = (""" You are a supervisor agent that can answer questions about the store's opening hours and the current date and time.
              You can use the following tools to answer the questions, using your own judgement:
              supervisor_agent: This tool provides the opening hours schedule of a shop.
              time_teller_agent: This tool provides the current date and time.
              You can use the tools in parallel""")

supervisor = create_supervisor(
    agents = [supervisor_agent, time_teller_agent],
    model=llm,
    output_mode="full_history",
    prompt=prompt,
)

chatAgent = supervisor.compile()

In [16]:
from langchain_core.messages import HumanMessage

user_input = input("Enter your query for the chat agent: ")

chatMessage = HumanMessage(user_input)

chatMessage.pretty_print()

result = chatAgent.invoke(chatMessage)

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

Hi


APIConnectionError: Connection error.